In [25]:
# Install required packages
!pip install torch-geometric
!pip install -q git+https://github.com/snap-stanford/deepsnap.git
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.3.0+cu121.html

  Preparing metadata (setup.py) ... done
Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html
  Using cached https://data.pyg.org/whl/torch-2.3.0%2Bcu121/torch_sparse-0.6.18%2Bpt23cu121-cp310-cp310-linux_x86_64.whl (5.1 MB)


In [26]:
import os
import torch
import torch_geometric
import torch.nn as nn
import torch.nn.functional as F

from torch import Tensor
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_sparse import SparseTensor, matmul
torch.__version__

OSError: /usr/local/lib/python3.10/dist-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

Reference:
  https://colab.research.google.com/drive/1KKugoFyUdydYC0XRyddcROzfQdMwDcnO?usp=sharing

Import data (initial embeddings)

In [ ]:
df = pd.read_csv()

In [ ]:
# Train test split
train, test = train_test_split(df.values, test_size = 0.2)
train = pd.DataFrame(train, columns = df.columns)
test = pd.DataFrame(test, columns = df.columns)

**LightGCN**
*   Adjacency matrix of an undirected bipartite graph to indicate the existence of interaction between user and item.

*   Normalized adjacency matrix $\tilde{A}$ as $$\tilde{A} = D^{-\frac{1}{2}} A D^{-\frac{1}{2}}$$

*   Final node embedding matrix: $$E^{(K)} = \tilde{A}^{(K)} E W$$

*   Multi-scale diffusion: $$\alpha_0 E^{(0)} + \alpha_1 E^{(1)} + \cdots + \alpha_k E^{(K)}$$
and for simplicity, LightGCN uses the uniform coefficient, i.e., $$\alpha_k = \frac{1}{K+1} \quad \text{for} \quad k = 0, \dots, K$$





In [ ]:
class LightGCNConv(MessagePassing):
  def __init__(self, num_users, num_items, embedding_dim, num_layers, add_self_loops=False):
    '''
    Args:
      num_users: number of users
      num_items: number of items
      embedding_dim: dimensionality of embeddings
      num_layers: number of message passing layers
      add_self_loops(optional): whether to add self
    '''
    super().__init__()
    self.num_users = num_users
    self.num_items = num_items
    self.embedding_dim = embedding_dim
    self.num_layers = num_layers
    self.add_self_loops = add_self_loops

    self.init_parameters()

  def init_embedding(self):
    # Substitute the initial embeddings here
    self.user_emb = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embedding_dim)
    self.item_emb = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embedding_dim)

    # Random normal initialization
    # (other options: uniform/normal Xavier initialization)
    nn.init.normal_(self.user_emb.weight, std=0.1)
    nn.init.normal_(self.item_emb.weight, std=0.1)

  def forward(self, edge_index: SparseTensor):
    # Compute normalized adjacency matrix
    A_tilde = gcn_norm(edge_index, add_self_loops=self.add_self_loops)

    # Concat the embeddings of users and items
    emb_0 = torch.cat([self.user_emb.weight, self.item_emb.weight])
    embs = [emb_0]

    # Multi-scale diffusion
    emb_k = emb_0
    for k in range(self.num_layers):
      emb_k = self.propagate(A_tilde, x=emb_k)
      embs.append(emb_k)

    embs = torch.stack(embs, dim=1)
    emb_final = torch.mean(embs, dim=1) # Uniform coefficient / mean

    # Split the embeddings to user embeddings and item embeddings
    user_emb_final, item_emb_final = torch.split(emb_final, [self.num_users, self.num_items])
    return user_emb_final, self.user_emb.weight, item_emb_final, self.item_emb.weight

  def message(self, x_j: Tensor) -> Tensor:
    return x_j

  def message_and_aggregate(self, adj: SparseTensor, x: Tensor) -> Tensor:
    return matmul(adj, x)


In [ ]:
model = LightGCNConv(num_users, num_items, 64, 3)

**Loss Function**
* We use Bayesian Personalized Ranking (BPR) loss here (a personalized surrogate
loss that aligns between with the recall@K metric)
$$LL_{\text{BPR}} = -\frac{1}{|U|} \sum_{u \in U} \frac{1}{|N_u|} \frac{1}{|N_u^c|} \sum_{i \in N_u} \sum_{j \notin N_u} \ln \sigma(f_{\theta}(u,i) - f_{\theta}(u,j))$$ where $f_{\theta}(u,v)$ is the score function

* Mini-batch training for the BPR loss

In [ ]:
def bpr_loss(user_emb_final, pos_item_emb_final, neg_item_emb_final):
  pos_scores = torch.mul(user_emb_final, pos_item_emb_final).sum(dim=-1)
  neg_scores = torch.mul(user_emb_final, neg_item_emb_final).sum(dim=-1)

  loss = -torch.mean(F.softplus(pos_scores - neg_scores))
  return loss

**Evaluation Metrics**
*   Recall@K = # of correctly recommended items in K / Total # of items the user interacted with
*   Precision@K = # of correctly recommended items in K / K




In [ ]:
def recall_precision_at_K(test_data, r, k):
  num_correct_rec = r[:, :k].sum(1)
  user_interacted = torch.Tensor([len(test_data[i]) for i in range(len(test_data))]) # number of items each user interacted with in test data
  recall_at_k = torch.mean(num_correct_rec / user_interacted)
  precision_at_k = torch.mean(num_correct_rec) / k
  return recall_at_k.item(), precision_at_k.item()